# Linear models with Scikit-learn

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import load_boston, load_iris, load_wine, load_digits, \
                             load_breast_cancer, load_diabetes, fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

## Boston

In [3]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
lm = LinearRegression()
lm.fit(X_train, y_train)
lm.score(X_test, y_test) # bounces around depending on test set

0.6567519760790881

In [20]:
x = [[0.03, 0, 7, 0, 0.47, 7.2, 61.1, 4.97, 2, 242, 17.8, 392, 4]]
lm.predict(x)

array([30.68955324])

In [ ]:
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    print(lm.score(X_test, y_test))

In [ ]:
# RF has more stable scores and generally better but linear model works well here
for i in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    rf = RandomForestRegressor(n_estimators=10)
    rf.fit(X_train, y_train)
    print(rf.score(X_test, y_test))

## NYC rent data set (Kaggle)

*Data use rules prevent us from storing this data in this repo*. Download the data
set from Kaggle. (You must be a registered Kaggle user and must be logged in.)
Go to the Kaggle [data page](https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/data)
and save `train.json`

In [ ]:
def load_rent():
    """

    :return:
    """
    df = pd.read_json('../../data/rent.json')

    # Create ideal numeric data set w/o outliers etc...
    df = df[(df.price > 1_000) & (df.price < 10_000)]
    df = df[df.bathrooms <= 4]  # There's almost no data for above with small sample
    df = df[(df.longitude != 0) | (df.latitude != 0)]
    df = df[(df['latitude'] > 40.55) & (df['latitude'] < 40.94) &
            (df['longitude'] > -74.1) & (df['longitude'] < -73.67)]
    df = df.sort_values('created')
    df_rent = df[['bedrooms', 'bathrooms', 'latitude', 'longitude', 'price']]

    return df_rent

In [ ]:
df_rent = load_rent()
X = df_rent.drop('price', axis=1)
y = df_rent['price']
X.head()

In [ ]:
def score(X, y, model, n=10):
    train_scores = []
    test_scores = []
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        model.fit(X_train, y_train)
        train_score = model.score(X_train, y_train)
        train_scores.append(train_score)
        test_score = model.score(X_test, y_test)
        test_scores.append(test_score)
    return np.mean(train_scores), np.mean(test_scores)

In [ ]:
score(X, y, LinearRegression())

### Random Forest much stronger than linear model

In [ ]:
score(X, y, RandomForestRegressor(n_estimators=10), n=5)

## Adding codependent and noise columns likely doesn't affect predictions

In [ ]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target

In [ ]:
# Add columns with 1..n
for i in range(100):
    X[f'ID{i}'] = range(1,len(X)+1)
# dup some columns
X['CRIM2'] = X['CRIM']
X['AGE2'] = X['AGE']
X.head()

In [ ]:
score(X, y, LinearRegression())